In [1]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
import pickle
import random

Category = "Country"
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Europe + NorthAmerica + SouthAmerica + Oceania
#countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']

w_length = 200

### Consolidate and pickle country files

In [2]:
for country in countriesOfInterest:
    if len(glob.glob("Raw Track Data\\" + country + ".p")) == 0 :
        hold = pd.DataFrame()
    else:
        hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    print(country, hold.shape)
    countryTracks = glob.glob("Raw Track Data\\" + country + "*.csv")
    print(country, len(countryTracks))
    for file in countryTracks:
        new = pd.read_csv(file)
        new["Country"] = file[15:17]
        new["Year"] = file[18:22]
        new["Playlist"] = file[23:-4]
        hold = hold.append(new)
        pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )
        os.remove(file)
    hold = hold.drop_duplicates(["track_id", "start"])
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

DZ (72570, 34)
DZ 0
EG (151231, 34)
EG 0
MA (82638, 34)
MA 0
ZA (1492095, 34)
ZA 0
TN (153437, 34)
TN 0
BH (16939, 34)
BH 0
HK (500127, 34)
HK 0
IN (1106, 34)
IN 0
ID (1173680, 34)
ID 0
IL (904243, 34)
IL 0
JP (1156764, 34)
JP 0
JO (31052, 34)
JO 0
KW (10846, 34)
KW 0
LB (674, 34)
LB 0
MY (501798, 34)
MY 0
OM (13487, 34)
OM 0
PS (15113, 34)
PS 0
PH (741723, 34)
PH 0
QA (7547, 34)
QA 0
SA (9520, 34)
SA 0
SG (469269, 34)
SG 0
TW (1650545, 34)
TW 0
TH (934511, 34)
TH 0
AE (67658, 34)
AE 11
VN (757004, 34)
VN 9
AD (8626, 34)
AD 0
AT (348710, 34)
AT 0
BE (578645, 34)
BE 0
BG (7768, 34)
BG 0
CY (0, 34)
CY 0
CZ (51422, 34)
CZ 0
DK (1692746, 34)
DK 0
EE (2106, 34)
EE 8
FI (1207800, 34)
FI 12
FR (942176, 34)
FR 10
DE (212712, 34)
DE 6
GR (0, 34)
GR 9
HU (0, 34)
HU 1
IS (269196, 34)
IS 0
IE (902724, 34)
IE 0
IT (621972, 34)
IT 0
LV (1966, 34)
LV 0
LI (0, 34)
LI 0
LT (0, 34)
LT 0
LU (1402, 34)
LU 0
MT (0, 34)
MT 0
MC (3215, 34)
MC 0
NL (885980, 34)
NL 0
NO (1083292, 34)
NO 0
PL (1025996, 34)
PL 0

In [3]:
hold

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,p1,p2,p3,p4,...,t7,t8,t9,t10,t11,t12,track_id,Country,Year,Playlist
681,0.00000,0.49914,0.000,-60.000,0.00000,-60.000,0.019,0.040,0.045,0.024,...,14.833,5.359,-27.228,0.973,-10.640,-7.228,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
682,0.49914,0.22644,1.000,-60.000,0.03370,-10.164,0.646,0.420,0.522,0.560,...,-20.233,-14.167,-6.882,34.476,41.585,7.220,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
683,0.72558,0.12181,0.381,-31.803,0.00782,-26.013,0.940,0.858,0.439,0.460,...,16.783,-18.108,13.722,32.195,-23.695,-3.539,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
684,0.84739,0.60340,0.952,-26.892,0.18692,-14.485,0.073,0.149,0.097,0.237,...,64.849,-42.428,21.399,50.362,-43.406,-18.990,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
685,1.45079,0.20980,0.775,-21.574,0.04262,-13.192,0.296,0.377,0.302,0.450,...,38.661,-2.748,52.698,13.987,-17.991,-5.808,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76256,226.15646,0.31379,0.176,-40.653,0.02574,-37.555,0.123,0.319,0.958,0.149,...,17.314,26.952,-44.420,-19.389,17.014,13.639,spotify:track:7wfZMooiq270lfOxJMPdbH,NZ,2019,37i9dQZF1DX0FJ8JYkqiJu
76257,226.47025,0.13351,0.390,-43.643,0.04982,-39.712,0.056,0.114,0.398,0.097,...,55.226,-1.846,-44.933,-21.813,-22.671,6.419,spotify:track:7wfZMooiq270lfOxJMPdbH,NZ,2019,37i9dQZF1DX0FJ8JYkqiJu
76258,226.60376,0.10971,0.128,-44.954,0.01964,-39.597,0.089,0.223,0.647,0.108,...,11.157,8.371,-50.210,-13.643,1.075,18.270,spotify:track:7wfZMooiq270lfOxJMPdbH,NZ,2019,37i9dQZF1DX0FJ8JYkqiJu
76259,226.71347,1.25401,0.081,-45.382,0.07637,-42.713,0.117,0.191,0.534,0.101,...,43.906,36.631,-58.818,-20.703,-26.776,16.040,spotify:track:7wfZMooiq270lfOxJMPdbH,NZ,2019,37i9dQZF1DX0FJ8JYkqiJu


### Remove non-unique tracks

In [4]:
allTracks = pd.DataFrame()
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[["Country", "Year", "track_id", "Playlist"]]
    allTracks = allTracks.append(hold)
allTracks

,Country,Year,track_id,Playlist
19557,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19558,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19559,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19560,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19561,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
...,...,...,...,...
76256,NZ,2019,spotify:track:7wfZMooiq270lfOxJMPdbH,37i9dQZF1DX0FJ8JYkqiJu
76257,NZ,2019,spotify:track:7wfZMooiq270lfOxJMPdbH,37i9dQZF1DX0FJ8JYkqiJu
76258,NZ,2019,spotify:track:7wfZMooiq270lfOxJMPdbH,37i9dQZF1DX0FJ8JYkqiJu
76259,NZ,2019,spotify:track:7wfZMooiq270lfOxJMPdbH,37i9dQZF1DX0FJ8JYkqiJu


In [5]:
hold = allTracks.groupby("track_id").nunique()
print(hold.shape[0], "Unique tracks per country")
keep = hold[hold.Country==1].index
allTracks = allTracks[allTracks.track_id.isin(keep)]
print(len(keep), "Unique tracks across countries")

42830 Unique tracks per country
41633 Unique tracks across countries


In [6]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[hold.track_id.isin(keep)]
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

### Create training, testing, and validation datasets

In [7]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    test = hold.loc[hold.track_id.isin(testTracks)]
    hold = hold.loc[~hold.track_id.isin(testTracks)]
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    val = hold.loc[hold.track_id.isin(testTracks)]
    train = hold.loc[~hold.track_id.isin(testTracks)]
    pickle.dump( train, open( "Raw Track Data\\" + country + "_train.p", "wb" ) )
    pickle.dump( test, open( "Raw Track Data\\" + country + "_test.p", "wb" ) )
    pickle.dump( val, open( "Raw Track Data\\" + country + "_val.p", "wb" ) )